IMPORTING LIBRARIES FOR CREATING MODEL

In [8]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.optimizers import SGD



CREATING DATASETS FOR TRAINING AND VALIDATION

In [9]:
training_datagen = ImageDataGenerator(rescale = 1./255,horizontal_flip=True,rotation_range=45,height_shift_range=0.2,fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory("fire_dataset",target_size=(224,224),class_mode='categorical',batch_size = 64)
validation_generator = validation_datagen.flow_from_directory("fire_dataset",target_size=(224,224),class_mode='categorical',batch_size= 16)

Found 999 images belonging to 2 classes.
Found 999 images belonging to 2 classes.


CREATING INCEPTIONV3 MODEL FOR IMAGE RECOGNITION AND CLASSIFICATION OF FIRE AND NONE FIRE IMAGE.
OUR MODEL CONTAINS 1 INPUT LAYER(GLOBALAVERAGEPOOLING2D), 3 DENSE LAYERS AND 3 DROPOUT AYERS AND 1 OUTPUT LAYER. 

In [10]:
input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 20,validation_data = validation_generator,validation_steps = 14)

#To train the top 2 inception blocks, freeze the first 249 layers and unfreeze the rest.
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True
#Recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator,steps_per_epoch = 14,epochs = 10,validation_data = validation_generator,validation_steps = 14)

Epoch 1/20
14/14 [==============================] - 39s 3s/step - loss: 6.0478 - acc: 0.6897 - val_loss: 0.3287 - val_acc: 0.7366
Epoch 2/20
14/14 [==============================] - 41s 3s/step - loss: 0.3046 - acc: 0.8634 - val_loss: 0.1648 - val_acc: 0.9464
Epoch 3/20
14/14 [==============================] - 44s 3s/step - loss: 0.3071 - acc: 0.8840 - val_loss: 0.1366 - val_acc: 0.9598
Epoch 4/20
14/14 [==============================] - 44s 3s/step - loss: 0.2253 - acc: 0.9242 - val_loss: 0.0907 - val_acc: 0.9777
Epoch 5/20
14/14 [==============================] - 43s 3s/step - loss: 0.2171 - acc: 0.9242 - val_loss: 0.1946 - val_acc: 0.9062
Epoch 6/20
14/14 [==============================] - 45s 3s/step - loss: 0.3661 - acc: 0.9036 - val_loss: 0.2093 - val_acc: 0.9509
Epoch 7/20
14/14 [==============================] - 45s 3s/step - loss: 0.1705 - acc: 0.9442 - val_loss: 0.1038 - val_acc: 0.9777
Epoch 8/20
14/14 [==============================] - 43s 3s/step - loss: 0.1427 - acc: 0.94

APPLICATION OF OUR MODEL ON LIVE FEED.

In [27]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from playsound import playsound
from keras.preprocessing import image
#Load the saved model
video = cv2.VideoCapture(0)
for i in range(100):
        _,frame = video.read()
#Convert the captured frame into RGB
        im = Image.fromarray(frame,'RGB')
#Resizing into 224x224 because we trained the model with this image size.
        im = im.resize((224,224))
        img_array = image.img_to_array(im)
        img_array = np.expand_dims(img_array, axis=0) / 255
        probabilities = model.predict(img_array)[0]
        #Calling the predict method on model to predict 'fire' on the image
        prediction = np.argmax(probabilities)
        #if prediction is 0, which means there is fire in the frame.
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        #if prediction == 0:
                
                #playsound('Fire alarm - 2018.mp3')
                #print('playing sound using  playsound')
        print(prediction)
        cv2.imshow("Capturing", frame)
        key=cv2.waitKey(1)
        if key == ord('q'):
                break
video.release()
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
